In [ ]:
import os
from pprint import pprint
path =  '/mnt/sdc/lanwei/IL8/v1'
score_line = {}
for root, dirs, files in os.walk(path):
    for dir in dirs:
        sub_path = os.path.join(root, dir)
        if 'log.txt' not in os.listdir(sub_path):
            continue
        log_file = open(sub_path + '/log.txt', 'r')
        if sum(1 for line in log_file) < 20:
            print(f'your docking job {dir} was failed,please cheak or rerun')
            continue
        log_file.seek(0)
        best_score = log_file.readlines()[14]
        if best_score == '':
            continue
        best_score_split = best_score.split()
        if best_score_split[0] != '1':
            print(f'cheak your {dir} logfiles, you score is not the best')
            continue
        score = best_score_split[1]
        
        score_line[dir] =  score
pprint(score_line)

min_scores  = {}
for dirs, scores in score_line.items():
    sequence = (dirs.split("-")[0])
    if sequence not in min_scores or float(scores) < float(min_scores[dirs]):
        min_scores[dirs] = scores
    